In [1]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgbm
#from xgboost import XGBRegressor
#import xgboost as xgb
#from catboost import CatBoostRegressor
#from lightgbm.sklearn import LGBMRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold

In [2]:
train_time = 1 * 60 * 60
objective = 'lgbmregressor'

In [3]:
sys.path.append(os.path.realpath('..'))

In [4]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
sample_sub = pd.read_csv(r"..\data\sample_submission.csv", index_col=0)

In [5]:
del train_df['Over18']
del train_df['EmployeeCount']
del train_df['StandardHours']

In [6]:
columns_to_vectorize = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
for vector_target in columns_to_vectorize:
    print(vector_target)
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

BusinessTravel
Department
EducationField
Gender
JobRole
MaritalStatus
OverTime


In [7]:
features = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction','HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
        'BusinessTravel_v', 'Department_v', 'EducationField_v','Gender_v', 'JobRole_v', 'MaritalStatus_v', 'OverTime_v']
target = ['Attrition']

In [8]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [9]:
RANDOM_STATE = 12 
FOLDS = 5
param_grid_history = {}

def render_model(param_grid):
    reg = lgbm.LGBMRegressor(**param_grid, metric='AUC', random_state=1)
    return reg
    
def train_model(reg, X_train, y_train, X_valid, y_valid):
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True)], verbose=False)

    preds = reg.predict(train_df[features])
    fpr, tpr, _ = roc_curve(train_df[target], preds)
    roc_auc = auc(fpr, tpr)
    return reg, roc_auc

def objective_v2(trial):
    scores = []
    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5),
        'max_depth': trial.suggest_int('max_depth', 3, 1000), 
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 100),
        'num_leaves': trial.suggest_int('num_leaves', 100, 10000),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.1, 1),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.1, 1),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.1, 5),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10), 
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 1000), 
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 100), 
        'n_estimators': trial.suggest_int('n_estimators', 10, 100000), 
    } 
        
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        print(10*"=", f"Fold={fold+1}", 10*"=")
        X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
        y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 
        
        reg = render_model(param_grid)
        reg, roc_auc = train_model(reg, X_train, y_train, X_valid, y_valid)

        scores.append(roc_auc)
    mean_scores = np.mean(scores)
    param_grid_history[mean_scores] = param_grid

    return mean_scores


In [10]:
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name=objective)
study.optimize(objective_v2, timeout=train_time)

[I 2023-01-17 22:33:53,911] A new study created in memory with name: lgbmregressor


========== Fold=1 ==========
[LightGBM] [Warning] feature_fraction is set=0.8160289228991924, colsample_bytree=0.16222481148852086 will be ignored. Current value: feature_fraction=0.8160289228991924
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=659 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_gain_to_split is set=0.8351399385880928, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.8351399385880928
[LightGBM] [Warning] bagging_fraction is set=0.8143801152823923, subsample=0.741313044938092 will be ignored. Current value: bagging_fraction=0.8143801152823923
[LightGBM] [Warning] lambda_l1 is set=0.536601065081237, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.536601065081237
[LightGBM] [Warning] lambda_l2 is set=0.7649328175185188, reg_lambda=8 will be ignored. Current value: lambda_l2=0.7649328175185188
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[4]	v

D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


========== Fold=2 ==========
[LightGBM] [Warning] feature_fraction is set=0.8160289228991924, colsample_bytree=0.16222481148852086 will be ignored. Current value: feature_fraction=0.8160289228991924
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=659 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] min_gain_to_split is set=0.8351399385880928, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.8351399385880928
[LightGBM] [Warning] bagging_fraction is set=0.8143801152823923, subsample=0.741313044938092 will be ignored. Current value: bagging_fraction=0.8143801152823923
[LightGBM] [Warning] lambda_l1 is set=0.536601065081237, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.536601065081237
[LightGBM] [Warning] lambda_l2 is set=0.7649328175185188, reg_lambda=8 will be ignored. Current value: lambda_l2=0.7649328175185188
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	v

D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


...

D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in 

Early stopping, best iteration is:
[17]	valid_0's auc: 0.833263
========== Fold=5 ==========
[LightGBM] [Warning] feature_fraction is set=0.5346398202583971, colsample_bytree=0.24668275443232263 will be ignored. Current value: feature_fraction=0.5346398202583971
[LightGBM] [Warning] min_data_in_leaf is set=26, min_child_samples=730 will be ignored. Current value: min_data_in_leaf=26
[LightGBM] [Warning] min_gain_to_split is set=0.4980431688212577, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.4980431688212577
[LightGBM] [Warning] bagging_fraction is set=0.8317622014050856, subsample=0.10223430367775788 will be ignored. Current value: bagging_fraction=0.8317622014050856
[LightGBM] [Warning] lambda_l1 is set=0.5781091410183, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5781091410183
[LightGBM] [Warning] lambda_l2 is set=0.8161074636810143, reg_lambda=59 will be ignored. Current value: lambda_l2=0.8161074636810143
Training until validation scores don'

D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in 

[LightGBM] [Warning] feature_fraction is set=0.5079008229495393, colsample_bytree=0.2900258039104026 will be ignored. Current value: feature_fraction=0.5079008229495393
[LightGBM] [Warning] min_data_in_leaf is set=29, min_child_samples=978 will be ignored. Current value: min_data_in_leaf=29
[LightGBM] [Warning] min_gain_to_split is set=0.5342360034180098, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.5342360034180098
[LightGBM] [Warning] bagging_fraction is set=0.6061152674907859, subsample=0.10425434319977325 will be ignored. Current value: bagging_fraction=0.6061152674907859
[LightGBM] [Warning] lambda_l1 is set=0.5961671971529413, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5961671971529413
[LightGBM] [Warning] lambda_l2 is set=0.8398197350937658, reg_lambda=63 will be ignored. Current value: lambda_l2=0.8398197350937658
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[5]	valid_0's auc: 0.801356


D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
[W 2023-01-17 22:34:03,443] Trial 17 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "D:\source\repos\venv\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_19772\2665844800.py", line 44, in objective_v2
    reg, roc_auc = train_model(reg, X_train, y_train, X_valid, y_valid)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_19772\2665844800.py", line 10, in train_model
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True)], 

In [ ]:
parm_grid_bytes = pickle.dumps(param_grid_history)
with open(f"rendered_data/{objective}_bytes.hex", "wb") as binary_file:
    binary_file.write(parm_grid_bytes)

In [ ]:
def train(param_grid):
    reg = render_model(param_grid)
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df[features], train_df[target])):
        print(10*"=", f"FINAL TRAINING Fold={fold+1}", 10*"=")
        X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
        y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 
        reg, roc_auc = train_model(reg, X_train, y_train, X_valid, y_valid)
    return reg

percent = math.ceil(len(param_grid_history.keys()) * .1)
top = sorted(list(param_grid_history.keys()))[-percent:]

train_preds = []
test_preds = []
for key in tqdm(top):
    model = train(param_grid_history[key])
    train_preds.append(model.predict(X))
    test_preds.append(model.predict(X_test))

train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

In [ ]:
train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
train_submission.to_csv(fr'rendered_data/{objective}_train_submission.csv', index=False)

test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
test_submission.to_csv(fr'rendered_data/{objective}_test_submission.csv', index=False)